#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [1]:

import sqlite3
from xmlrpc.server import SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCServer


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)


server = SimpleXMLRPCServer(("localhost", 8038),
                            requestHandler=RequestHandler)

LOG_DB = 'resources/log.db'


# Добавление строки в лог
def add_log(event, event_time, work_time):
    conn = sqlite3.connect(LOG_DB)
    conn.cursor().execute('INSERT INTO log VALUES (?, ?, ?)', (event, event_time, work_time))
    conn.commit()
    conn.close()
    return True


server.register_function(add_log, 'add_log')


def get_conditions_and_params(par_type='', par_start_time='', par_end_time='', max_eval_time=None):
    conditions = []
    params = {}
    if par_type != '':
        conditions.append('log.event_type = :par_type')
        params['par_type'] = par_type
    if par_start_time != '':
        conditions.append('log.event_time >= :par_start_time')
        params['par_start_time'] = par_start_time
    if par_end_time != '':
        conditions.append('log.event_time <= :par_end_time')
        params['par_end_time'] = par_end_time
    if not max_eval_time is None:
        conditions.append('log.work_time <= :max_eval_time')
        params['max_eval_time'] = max_eval_time
    return conditions, params


def get_info_from_log(par_type='', par_start_time='', par_end_time='', max_eval_time=None):
    par_type = par_type.strip()
    par_start_time = par_start_time.strip()
    par_end_time = par_end_time.strip()

    conn = sqlite3.connect('resources/log.db')  #LOG_DB)
    conditions, params = get_conditions_and_params(par_type, par_start_time, par_end_time, max_eval_time)
    query = 'SELECT * FROM log' + (' WHERE ' if len(conditions) > 0 else '') + ' AND '.join(conditions)
    cursor = conn.execute(query, params)
    info = "".join([str(row) + "\n" for row in cursor])
    conn.close()
    return info
server.register_function(get_info_from_log, 'get_info_from_log')

def create_db():
    conn = sqlite3.connect(LOG_DB)
    conn.cursor().execute('''CREATE TABLE IF NOT EXISTS log (event_type TEXT, event_time TEXT, work_time REAL)''')
    conn.commit()
    conn.close()

create_db()
print("Listening on port 8038...")
server.serve_forever()


Listening on port 8038...


127.0.0.1 - - [11/Oct/2023 12:42:32] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 12:42:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [11/Oct/2023 12:42:36] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 